In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from scipy.stats import skew
import pingouin as pg

import hotspot_util

***
### Load one GetisOrd Stats CSV, specify `dpath`

In [ ]:
dpath = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Summary_Analysis/Hotspot_Data/WT/Mid_Striatum/Adult_Mid_striatum/CtxStrRGP21F3_sectionB3L_CalB_DAPI_20x20_GetisOrdStats copy.csv'
data = pd.read_csv(dpath)
data.head()

### Plot cumulative histogram of Z-Scores

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Z-Score'], nbinsx=300,  cumulative_enabled=True, histnorm='probability', marker_color='steelblue'))
fig.update_layout(template='simple_white', width=800, height=400, xaxis_title='Z-Score', yaxis_title='Frequency')
fig.show()

### Plot distribution of Z-scores

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data['Z-Score'], nbinsx=300,  cumulative_enabled=False, histnorm='probability', marker_color='steelblue'))
fig.update_layout(template='simple_white', width=800, height=400, xaxis_title='Z-Score', yaxis_title='Frequency',
                  title_text= '{}<br>Skew: {}'.format(os.path.basename(dpath), np.round(skew(data['Z-Score']), 3)))
fig.show()

### Create distribution plot and CDF plot and save both

In [ ]:
# make distplot and save
fig = ff.create_distplot([data['Z-Score']], ['Data'], bin_size=1, colors=['steelblue'], show_hist=True, show_curve=True, show_rug=False)
fig.update_layout(width=800, height=400, template='simple_white', dragmode='pan', showlegend=False, xaxis_title='Z-Score', yaxis_title='Frequency',
                  title_text= '{}<br>Skew: {}'.format(os.path.basename(dpath), np.round(skew(data['Z-Score']), 3)))
fig.show(config={'scrollZoom':True})
dist_folder_path = '/Users/joezaki/Desktop/Hotspot_Dists/{}'.format(os.path.dirname(dpath).split('/')[-1])
if not os.path.exists(dist_folder_path):
    os.makedirs(dist_folder_path)
fig.write_image(os.path.join(dist_folder_path, '{}_Dist.png'.format(os.path.basename(dpath).split('.')[0])))


# make CDF plot and save
fig = px.ecdf(data, x="Z-Score")
fig.update_layout(template='simple_white', width=800, height=400, xaxis_title='Z-Score', yaxis_title='Frequency')
fig.update_layout(width=800, height=400, template='simple_white', dragmode='pan', showlegend=False, xaxis_title='Z-Score', yaxis_title='Frequency',
                  title_text= '{}<br>Skew: {}'.format(os.path.basename(dpath), np.round(skew(data['Z-Score']), 3)))
fig.show()
cdf_folder_path = '/Users/joezaki/Desktop/Hotspot_CDF/{}'.format(os.path.dirname(dpath).split('/')[-1])
if not os.path.exists(cdf_folder_path):
    os.makedirs(cdf_folder_path)
fig.write_image(os.path.join(cdf_folder_path, '{}_CDF.png'.format(os.path.basename(dpath).split('.')[0])))

***
### Loop through a folder, make distribution and CDF plots for each file, and save them. Specify `folder_path`

In [ ]:
wt_or_cko = 'WT' # one of 'WT' or 'Rostral'
rostral_or_mid = 'Rostral' # one of 'Rostral' or 'Mid'
age = 'Adult' # one of 'P7', 'P21', or 'Adult'
folder_path = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Summary_Analysis/Hotspot_Data/{}/{}_Striatum/Adult_Rostral_Striatum/'.format(wt_or_cko, rostral_or_mid, age, rostral_or_mid)

data_files = np.array(os.listdir(folder_path))
is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
data_files = data_files[is_csv]

for data_file in data_files:
    dpath = os.path.join(folder_path, data_file)
    data = pd.read_csv(dpath)

    # make distplot and save
    fig = ff.create_distplot([data['Z-Score']], ['Data'], bin_size=1, colors=['steelblue'], show_hist=True, show_curve=True, show_rug=False)
    fig.update_layout(width=800, height=400, template='simple_white', dragmode='pan', showlegend=False, xaxis_title='Z-Score', yaxis_title='Frequency',
                    title_text= '{}<br>Skew: {}'.format(os.path.basename(dpath), np.round(skew(data['Z-Score']), 3)))
    fig.show(config={'scrollZoom':True})
    dist_folder_path = '/Users/joezaki/Desktop/Hotspot_Dists/{}'.format(os.path.dirname(dpath).split('/')[-1])
    if not os.path.exists(dist_folder_path):
        os.makedirs(dist_folder_path)
    fig.write_image(os.path.join(dist_folder_path, '{}_Dist.png'.format(os.path.basename(dpath).split('.')[0])))

    # make CDF plot and save
    fig = px.ecdf(data, x="Z-Score")
    fig.update_layout(template='simple_white', width=800, height=400, xaxis_title='Z-Score', yaxis_title='Frequency')
    fig.update_layout(width=800, height=400, template='simple_white', dragmode='pan', showlegend=False, xaxis_title='Z-Score', yaxis_title='Frequency',
                    title_text= '{}<br>Skew: {}'.format(os.path.basename(dpath), np.round(skew(data['Z-Score']), 3)))
    fig.show()
    cdf_folder_path = '/Users/joezaki/Desktop/Hotspot_CDF/{}'.format(os.path.dirname(dpath).split('/')[-1])
    if not os.path.exists(cdf_folder_path):
        os.makedirs(cdf_folder_path)
    fig.write_image(os.path.join(cdf_folder_path, '{}_CDF.png'.format(os.path.basename(dpath).split('.')[0])))

### Loop through the above folder and get the average z-score, the skew, and the variance for each image

In [ ]:
data_files = np.array(os.listdir(folder_path))
is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
data_files = data_files[is_csv]

agg_data = pd.DataFrame(columns=['Filename', 'Mean', 'SD', 'Skew'])

for data_file in data_files:
    dpath = os.path.join(folder_path, data_file)
    data = pd.read_csv(dpath)

    agg_data = agg_data.append(pd.Series({'Filename':os.path.basename(dpath).split('.')[0],
                                          'Mean':data['Z-Score'].mean(),
                                          'SD':data['Z-Score'].std(),
                                          'Skew':skew(data['Z-Score'])}), ignore_index=True)
agg_data['blank'] = ''

In [ ]:
summary_save_path = '/Users/joezaki/Desktop/Hotspot_SummaryStats'
summary_save_path = os.path.join(summary_save_path, os.path.dirname(dpath).split('/')[-1])
if not os.path.exists(summary_save_path):
    os.makedirs(summary_save_path)

for plot_var in ['Mean', 'SD', 'Skew']:
    save_path = os.path.join(summary_save_path, '{}.png'.format(plot_var))
    hotspot_util.plotMeanData(agg_data=agg_data, groupby='blank', plot_var=plot_var, datapoint_var='Filename', plot_title='',
                              colors=['steelblue'], plot_datapoints=True, y_title=plot_var, plot_width=250, save_path=save_path)

### Make gridded plot for distributions and CDFs for the above folder and save plot

In [ ]:
data_files = np.array(os.listdir(folder_path))
is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
data_files = data_files[is_csv]

cols = 3
rows = int(np.ceil(len(data_files) / cols))
subplot_titles = [x.split('_Getis')[0] for x in data_files]
fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles, shared_xaxes=False, horizontal_spacing=0.1, vertical_spacing=0.1)

for i, data_file in enumerate(data_files):
    dpath = os.path.join(folder_path, data_file)
    data = pd.read_csv(dpath)

    row = int(np.floor(i / cols)) + 1
    col = int(i % cols) + 1

    distplot_fig = ff.create_distplot([data['Z-Score']], ['Data'], bin_size=1, colors=['steelblue'], show_hist=True, show_curve=True, show_rug=False)

    fig.add_trace(distplot_fig['data'][0], row=row, col=col)
    fig.add_trace(distplot_fig['data'][1], row=row, col=col)
    fig.update_xaxes(title_text='Z-Score', row=row, col=col)
    fig.update_yaxes(title_text='Frequency', row=row, col=col)

subplot_size = 400
fig.update_layout(height=rows*subplot_size, width=cols*subplot_size, template='simple_white', dragmode='pan', showlegend=False,
                  xaxis_title='Z-Score', yaxis_title='Frequency', title_text=os.path.dirname(folder_path).split('/')[-1])
fig.update_yaxes(matches='y')
fig.update_xaxes(matches='x')
fig.update_annotations(font_size=12)
fig.show()
fig.write_image(os.path.join(summary_save_path, 'Dist_Plots.png'))

In [ ]:
data_files = np.array(os.listdir(folder_path))
is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
data_files = data_files[is_csv]

cols = 3
rows = int(np.ceil(len(data_files) / cols))
subplot_titles = [x.split('_Getis')[0] for x in data_files]
fig = make_subplots(rows=rows, cols=cols, subplot_titles=subplot_titles, shared_xaxes=False, horizontal_spacing=0.1, vertical_spacing=0.1)

for i, data_file in enumerate(data_files):
    dpath = os.path.join(folder_path, data_file)
    data = pd.read_csv(dpath)

    row = int(np.floor(i / cols)) + 1
    col = int(i % cols) + 1

    cdf_fig = px.ecdf(data, x="Z-Score")

    fig.add_trace(cdf_fig['data'][0], row=row, col=col)
    fig.update_xaxes(title_text='Z-Score', row=row, col=col)
    fig.update_yaxes(title_text='Frequency', row=row, col=col)

subplot_size = 400
fig.update_layout(height=rows*subplot_size, width=cols*subplot_size, template='simple_white', dragmode='pan', showlegend=False,
                  xaxis_title='Z-Score', yaxis_title='Frequency', title_text=os.path.dirname(folder_path).split('/')[-1])
fig.update_yaxes(matches='y')
# fig.update_xaxes(matches='x')
fig.update_annotations(font_size=12)
fig.show()
fig.write_image(os.path.join(summary_save_path, 'CDF_Plots.png'))

***
### Load data for three ages (P7, P21, Adult) and plot average skew across ages

In [ ]:
overall_dpath = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Summary_Analysis/Hotspot_Data/WT/'

all_ages_data_dict = {'P7':{'Rostral':os.path.join(overall_dpath, 'Rostral_Striatum/P7_Rostral_striatum/'),
                            'Mid'    :os.path.join(overall_dpath, 'Mid_Striatum/P7_Mid_striatum/')},
                      'P21':{'Rostral':os.path.join(overall_dpath, 'Rostral_Striatum/P21_Rostral_striatum/'),
                             'Mid'    :os.path.join(overall_dpath, 'Mid_Striatum/P21_Mid_striatum/')},
                      'Adult':{'Rostral':os.path.join(overall_dpath, 'Rostral_Striatum/Adult_Rostral_striatum/'),
                               'Mid'    :os.path.join(overall_dpath, 'Mid_Striatum/Adult_Mid_striatum/')}}

In [ ]:
colnames = ['Mouse', 'PartOfStriatum', 'Age', 'Filename', 'Mean', 'SD', 'Skew']
all_ages_data = pd.DataFrame(columns=colnames)

for age, age_path in all_ages_data_dict.items():
    for rostral_or_mid, folder_path in age_path.items():
        data_files = np.array(os.listdir(folder_path))
        is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
        data_files = data_files[is_csv]

        age_data = pd.DataFrame(columns=colnames)

        for data_file in data_files:
            dpath = os.path.join(folder_path, data_file)
            data = pd.read_csv(dpath)
            mouse = data_file.split('_section')[0]
            age_data = age_data.append(pd.Series({'Mouse':mouse,
                                                'PartOfStriatum':rostral_or_mid,
                                                'Age':age,
                                                'Filename':os.path.basename(dpath).split('.')[0],
                                                'Mean':data['Z-Score'].mean(),
                                                'SD':data['Z-Score'].std(),
                                                'Skew':skew(data['Z-Score'])}), ignore_index=True)
        
        all_ages_data = all_ages_data.append(age_data)

all_ages_data['Age'] = pd.Categorical(all_ages_data['Age'], categories=['P7', 'P21', 'Adult'])
all_ages_data['PartOfStriatum'] = pd.Categorical(all_ages_data['PartOfStriatum'])

In [ ]:
agg_data = all_ages_data.copy()
groupby = 'Age'
plot_var = 'Skew'
colors = ['black', 'skyblue', 'lightgrey']
hotspot_util.plotMeanData(agg_data=agg_data, groupby=groupby, plot_var=plot_var, colors=colors, datapoint_var='Filename',
                          y_title=plot_var, plot_datapoints=True, plot_title=rostral_or_mid)
pg.anova(data=agg_data, dv='Skew', between='Age')
# pg.pairwise_tests(data=agg_data, dv='Skew', between='Age', padjust='bh')

In [ ]:
agg_data = all_ages_data.copy()

groupby = 'Age'
separateby = 'PartOfStriatum'
colors = ['black', 'skyblue', 'lightgrey']

# groupby = 'PartOfStriatum'
# separateby = 'Age'
# colors = ['slategrey', 'slategrey']

plot_var = 'Skew'
datapoint_var = 'Mouse'
hotspot_util.plotAcrossGroups(agg_data=agg_data, groupby=groupby, separateby=separateby, plot_var=plot_var, datapoint_var=datapoint_var,
                              plot_datalines=False, plot_datapoints=True, y_title=plot_var, colors=colors, title='')
pg.mixed_anova(data=agg_data, dv=plot_var, between='Age', within='PartOfStriatum', subject='Mouse')
# pg.pairwise_tests(data=agg_data, dv=plot_var, between='Age', within='PartOfStriatum', subject='Mouse', within_first=True, padjust='bh')

### Compare skew of WT vs cKO at adulthood

In [ ]:
overall_dpath =  '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Summary_Analysis/Hotspot_Data/'

wt_paths  = {'Mid'     : os.path.join(overall_dpath, 'WT/Mid_Striatum/Adult_Mid_Striatum/'),
             'Rostral' : os.path.join(overall_dpath, 'WT/Rostral_Striatum/Adult_Rostral_Striatum/')}
cko_paths = {'Mid'     : os.path.join(overall_dpath, 'cKO/Mid_Striatum/Adult_Mid_Striatum/'),
             'Rostral' : os.path.join(overall_dpath, 'cKO/Rostral_Striatum/Adult_Rostral_Striatum/')}
group_paths = {'WT':wt_paths,
               'cKO':cko_paths}

colnames = ['Mouse', 'Group', 'PartOfStriatum', 'Filename', 'Mean', 'SD', 'Skew']
wt_cko_df = pd.DataFrame(columns=colnames)


for group, path_dict in group_paths.items():

    for part_of_striatum, folder_path in path_dict.items():
        
        data_files = np.array(os.listdir(folder_path))
        is_csv = [x.split('.')[-1] == 'csv' for x in data_files]
        data_files = data_files[is_csv]

        for data_file in data_files:
            dpath = os.path.join(folder_path, data_file)
            data = pd.read_csv(dpath)
            mouse = data_file.split('_section')[0]
            wt_cko_df = wt_cko_df.append(pd.Series({'Mouse':mouse,
                                                    'Group':group,
                                                    'PartOfStriatum':part_of_striatum,
                                                    'Filename':os.path.basename(dpath).split('.')[0],
                                                    'Mean':data['Z-Score'].mean(),
                                                    'SD':data['Z-Score'].std(),
                                                    'Skew':skew(data['Z-Score'])}), ignore_index=True)

wt_cko_df['Group'] = pd.Categorical(wt_cko_df['Group'], categories=['WT', 'cKO'])
wt_cko_df['PartOfStriatum'] = pd.Categorical(wt_cko_df['PartOfStriatum'], categories=['Rostral', 'Mid'])

In [ ]:
agg_data = wt_cko_df.copy()
groupby = 'Group'
plot_var = 'Mean'
colors = ['black', 'purple']
hotspot_util.plotMeanData(agg_data=wt_cko_df, groupby=groupby, plot_var=plot_var, plot_datapoints=True, datapoint_var='Mouse', y_title=plot_var, colors=colors)
pg.anova(data=agg_data, dv=plot_var, between=groupby)

In [ ]:
agg_data = wt_cko_df.copy()
groupby = 'Group'
separateby = 'PartOfStriatum'

groupby = 'PartOfStriatum'
separateby = 'Group'

plot_var = 'Mean'
colors = ['black', 'purple']
datapoint_var = 'Mouse'
hotspot_util.plotAcrossGroups(agg_data=wt_cko_df, groupby=groupby, separateby=separateby, plot_var=plot_var, datapoint_var=datapoint_var,
                              plot_datalines=True, plot_datapoints=False, y_title=plot_var, colors=colors, title='')
pg.mixed_anova(data=agg_data, dv=plot_var, between='Group', within='PartOfStriatum', subject='Mouse')